In [1]:
from typing import List
from Dataset.CustomDataset import AgeGroupAndAgeDataset, StandardDataset, AgeDatasetKL
from Dataset.CustomDataLoaders import CustomDataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from Utils import AAR, CSVUtils, AgeConversion
from Utils.Validator import Validator

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#########################
import pandas as pd
df = CSVUtils.get_df_from_csv("./training_caip_contest.csv", "./training_caip_contest/")
df_train, df_val = train_test_split(df, test_size=0.25, random_state=42)
aug = CSVUtils.get_df_from_csv("./augumentation_balanced_remove.csv", "./newAugmentationDataset/")
df_train_aug = pd.concat([df_train, aug], ignore_index=True)
df_train_aug = df_train_aug.reset_index(drop=True)
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
#########################

from torchvision import transforms
import torch
import numpy as np

transform_func = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.1, 0.1, 0.1, 0.1),
    transforms.RandAugment(2, 9),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

transform_func_val = transforms.Compose([
    transforms.Resize(224),
    transforms.PILToTensor(),
    transforms.ConvertImageDtype(torch.float),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])

# Implementazione di un Dataset utilizzando "CustomDataset" per l'architettura con Film
cd_train = AgeDatasetKL(df_train, path_col="path", label_col="age", label_function="Linear", 
                        transform_func=transform_func)
# Implementazione di un Dataset utilizzando "CustomDataset" per l'architettura con Film
cd_train_balanced = AgeDatasetKL(df_train_aug, path_col="path", label_col="age", label_function="Linear", 
                                transform_func=transform_func)
cd_train.set_n_classes(81)
cd_train.set_starting_class(1)
cd_train_balanced.set_n_classes(81)
cd_train_balanced.set_starting_class(1)
dm_train = CustomDataLoader(cd_train)
dm_train_balanced = CustomDataLoader(cd_train_balanced)
dl_train = dm_train.get_unbalanced_dataloader(batch_size=64, shuffle=True, drop_last=True, num_workers=4, prefetch_factor=2)
dl_train_balanced, sampler = dm_train_balanced.get_balanced_class_dataloader2(class_ranges=[(0, 11), (11, 21), (21, 31), (31, 41), (41, 51), (51, 61), (61, 71), (71, 91)], 
                                                                            batch_size=64, num_workers=4, prefetch_factor=2)
sampler.n_batches = len(dl_train)
sampler.p = np.array([0.23, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11, 0.11])

cd_val = StandardDataset(df_val, path_col="path", label_col="age", label_function="CAE", transform_func=transform_func_val)
cd_val.set_n_classes(81)
cd_val.set_starting_class(1)
validator = Validator(cd_val, AgeConversion.EVAge, 32, num_workers=4, prefetch_factor=2)

In [3]:
from ResNetFilmed.resnet import ResNetFiLMed, BackBone, ResNetNotFiLMed, DoNothingLayer
from torchvision.models import resnet18, ResNet18_Weights, efficientnet_b0, EfficientNet_B0_Weights
import torch
from torch import optim
import torch.nn.functional as F
from torch import nn

####################################################
EPOCHS = 24
####################################################

backbone = resnet18(ResNet18_Weights.IMAGENET1K_V1)
backbone.fc = DoNothingLayer()
backbone.train()
backbone.requires_grad_(True)
backbone.to("cuda")
model_age = ResNetNotFiLMed(backbone, 81)
model_age.load_state_dict(torch.load("./model_age_classification_simple.pt", map_location="cuda:0"))
opt = optim.SGD(set([*backbone.parameters(), *model_age.fc0.parameters()]), lr=1e-2, weight_decay=5e-5)
scheduler = optim.lr_scheduler.OneCycleLR(opt, max_lr=1e-2, steps_per_epoch=len(dl_train), epochs=EPOCHS, three_phase=True)
kl = nn.KLDivLoss(reduction="batchmean")

c:\Users\Gio\.conda\envs\AV_project\lib\site-packages\torchvision\models\_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


In [4]:
def forward_function(x):
    out = model_age(x)
    out = F.softmax(out, dim=-1)
    return out

ae, mae, val_aar, val_aar_old = validator.validate_ext2(forward_function)
print(ae, mae, val_aar, val_aar_old)

  3%|▎         | 150/4493 [00:18<04:08, 17.45 batch/s] 

In [ ]:
import numpy as np
def get_centers(outs, ys, old):
    centers_for_age = {x: [] for x in range(81)}

    for out, y in zip(outs, ys):
        for _out, _y in zip(out, y):
            centers_for_age[int(_y)].append(_out.detach().cpu().numpy())

    for i in centers_for_age:
        if len(centers_for_age[i]) > 0:
            centers_for_age[i] = torch.tensor(np.array(centers_for_age[i])).mean(dim=0).to("cuda")
        else:
            centers_for_age[i] = old[i]
    return centers_for_age

def update_centers(old, new, alpha=0.5):
    for i in new:
        new[i] = new[i] - alpha*(new[i] - old[i])
    return new

def get_centers_loss(out, y, centers_for_age):
    loss = None
    for _out, _y in zip(out, y):
        if loss is None:
            loss = torch.mean(torch.square(_out - centers_for_age[int(_y)]))
        else:    
            loss += torch.mean(torch.square(_out - centers_for_age[int(_y)]))
    return loss.to("cuda")

In [ ]:
centers_for_age = {x: torch.zeros(size=(512,), device="cuda") for x in range(81)}
best_val_aar = val_aar

for e in range(EPOCHS):
    with tqdm(zip(dl_train, dl_train_balanced), unit=" batch") as tepoch:
        for batch, batch_balanced in tepoch:
            opt.zero_grad()
            x, y = batch
            x_bal, y_bal = batch_balanced

            x = x.to("cuda")
            y_age: torch.Tensor = y[0].to("cuda")
            y_age_kl: torch.Tensor = y[1].to("cuda")

            x_bal = x_bal.to("cuda")
            y_age_bal: torch.Tensor = y_bal[0].to("cuda")
            y_age_kl_bal: torch.Tensor = y_bal[1].to("cuda")

            out_rep, out_age = model_age.forward_with_repr(x)
            loss_age_kl: torch.Tensor = kl(F.log_softmax(out_age, dim=-1), y_age_kl)
            out_age = F.softmax(out_age, dim=-1)
            out = AgeConversion.EVAge(out_age).to("cuda")
            loss_age = torch.mean(torch.abs(y_age - out))
            loss = loss_age_kl + loss_age

            out_rep_bal, out_age_bal = model_age.forward_with_repr(x_bal)
            loss_age_kl_bal: torch.Tensor = kl(F.log_softmax(out_age_bal, dim=-1), y_age_kl_bal)
            out_age_bal = F.softmax(out_age_bal, dim=-1)
            out_bal = AgeConversion.EVAge(out_age_bal).to("cuda")
            loss_age_bal = torch.mean(torch.abs(y_age_bal - out_bal))
            loss_bal = loss_age_kl_bal + torch.square(loss_age_bal - 2.0)

            loss_repr = get_centers_loss(out_rep, y_age, centers_for_age)
            loss_repr_bal = get_centers_loss(out_rep_bal, y_age_bal, centers_for_age)

            total_loss = loss + loss_bal + loss_repr + loss_repr_bal

            total_loss.backward()
            opt.step()
            scheduler.step()

            centers_for_age = update_centers(centers_for_age, get_centers((out_rep, out_rep_bal), (y_age, y_age_bal), centers_for_age), alpha=0.5)

            tepoch.set_postfix(loss_age_kl=loss_age_kl.detach().cpu().numpy(), loss_age=loss_age.detach().cpu().numpy(),
                                loss_age_kl_bal=loss_age_kl_bal.detach().cpu().numpy(), loss_age_bal=loss_age_bal.detach().cpu().numpy(),
                                loss_repr=loss_repr.detach().cpu().numpy(), loss_repr_bal=loss_repr_bal.detach().cpu().numpy(),
                                total_loss=total_loss.detach().cpu().numpy())

    def forward_function(x):
        out = model_age(x)
        out = F.softmax(out, dim=-1)
        return out
 
    ae, mae_, val_aar, val_aar_old = validator.validate_ext2(forward_function)
    print(ae, mae_, val_aar, val_aar_old)

    if best_val_aar < val_aar:
        best_val_aar = val_aar
        torch.save(model_age.state_dict(), "./model_age_feature_simple_no_loss.pt")
        print("Saved model")

6739 batch [34:31,  3.25 batch/s, loss_age=1.8559341, loss_age_bal=2.1530418, loss_age_kl=1.0525734314640984, loss_age_kl_bal=1.0539909581164602, loss_repr=0.25880098, loss_repr_bal=0.3337046, total_loss=2.7432467397690523] 
100%|██████████| 4493/4493 [03:06<00:00, 24.09 batch/s]


{0: tensor(7.6679, dtype=torch.float64), 1: tensor(2.8715, dtype=torch.float64), 2: tensor(1.8302, dtype=torch.float64), 3: tensor(2.1703, dtype=torch.float64), 4: tensor(2.3448, dtype=torch.float64), 5: tensor(2.1477, dtype=torch.float64), 6: tensor(2.6364, dtype=torch.float64), 7: tensor(4.2838, dtype=torch.float64)} tensor(2.2040, dtype=torch.float64) tensor(4.6650, dtype=torch.float64) tensor(5.7051, dtype=torch.float64)
